In [45]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
from gsmmutils.model.COBRAmodel import MyModel, check_balance
from cobra.flux_analysis import find_blocked_reactions, pfba, flux_variability_analysis
from cobra.io import write_sbml_model, read_sbml_model
from gsmmutils.omics.omics_integration import OmicsIntegration 
from gsmmutils.utils.utils import flux_change
import matplotlib.pyplot as plt
import pandas as pd
from os.path import join
from mewpy.omics import eFlux,ExpressionSet
import math
import numpy as np
from cobra.sampling import ACHRSampler
RESULTS_PATH = "/home/ecunha/omics-integration/results/ngaditana/PRJNA589063/"
DATA_PATH = "/home/ecunha/omics-integration/data/ngaditana/PRJNA589063"

In [11]:
model = MyModel(join(DATA_PATH, "../models/model_ng.xml"), "e_Biomass__cytop")
omics = OmicsIntegration(join(DATA_PATH,'raw_data.tsv'), model=model)
omics.counts.index = pd.Index([e.replace("-", "_").replace("_1", "") for e in omics.counts.index])
omics.data.index = pd.Index([e.replace("-", "_").replace("_1", "") for e in omics.data.index])
omics.drop_sample("C3_4")

Loading

Reactions: 4495
Metabolites: 3804
Genes: 1335
Model loaded


In [12]:
omics.getmm = pd.read_csv(join(DATA_PATH, 'getmm_with_replicates.tsv'), sep="\t", index_col=0)
omics.getmm.head()

,C3_1,C3_2,C3_3,C5_1,C5_2,C5_3,C5_4,N3_1,N3_2,N3_3,...,N5_3,N5_4,P3_1,P3_2,P3_3,P3_4,P5_1,P5_2,P5_3,P5_4
Ng00001_RA,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.873897,0.000000,...,0.000000,0.000000,0.000000,0.000000,2.868909,0.000000,0.000000,0.000000,0.0,0.0
Ng00002_RA,0.0,9.297582,8.242249,0.0,18.796616,0.0,28.357177,6.634684,12.938815,6.086297,...,5.959431,5.699995,26.462049,8.814936,0.000000,17.814291,18.289165,38.367042,0.0,0.0
Ng00003_RA,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
Ng00004_RA,0.0,0.000000,1.613867,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
Ng00005_RA,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0


In [13]:
omics.sum_tech_reps()
omics_data = omics.getmm.applymap(lambda x: math.log2(x + 1))

In [14]:
omics_data.head()

,C3,C5,N3,N5,P3,P5
Ng00001_RA,0.000000,0.000000,3.086714,0.000000,3.641041,0.000000
Ng00002_RA,6.152981,7.566931,7.250133,7.152946,7.737180,7.830514
Ng00003_RA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Ng00004_RA,2.898299,0.000000,0.000000,0.000000,0.000000,0.000000
Ng00005_RA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [37]:
expression = omics_data["C3"].to_numpy()[:, np.newaxis]
set_expression = ExpressionSet(omics_data.index.to_list(), conditions=["C3"], expression=expression)
c3 = eFlux(model, set_expression, parsimoneous=True)

Read LP format model from file /tmp/tmphwj0y6r8.lp
Reading time = 0.06 seconds
: 3804 rows, 8990 columns, 38916 nonzeros


In [38]:
expression = omics_data["P3"].to_numpy()[:, np.newaxis]
set_expression = ExpressionSet(omics_data.index.to_list(), conditions=["P3"], expression=expression)
p3 = eFlux(model, set_expression, parsimoneous=True)

Read LP format model from file /tmp/tmp2qkwbcvs.lp
Reading time = 0.07 seconds
: 3804 rows, 8990 columns, 38916 nonzeros


In [82]:
expression = omics_data["N5"].to_numpy()[:, np.newaxis]
set_expression = ExpressionSet(omics_data.index.to_list(), conditions=["N5"], expression=expression)
n5 = eFlux(model, set_expression, parsimoneous=True)

Read LP format model from file /tmp/tmpngeotugf.lp
Reading time = 0.06 seconds
: 3804 rows, 8990 columns, 38916 nonzeros


In [83]:
n5.fluxes['e_Biomass__cytop']

0.0016953855438577134

In [40]:
p3.fluxes['e_Biomass__cytop']

0.001807919355401335

In [44]:
p3.get_constraints()

OrderedDict([('T_O2tm__mitmem', (-1, 1)),
             ('BMGR6355__er', (0, 0.7210739229398475)),
             ('BMGR0494__mito', (0, 0.674264173616979)),
             ('BMGR6010__nucl', (0, 0.6770463921712977)),
             ('R01652__cytop', (0, 0.675126051579233)),
             ('TO1101605t__plas', (0, 0.6345086563867199)),
             ('T_gGbCAt__plas', (-1, 1)),
             ('R00243__mito', (-0.7761031200822173, 0.7761031200822173)),
             ('DGTSS_205_205__er', (0, 0.6848632491305116)),
             ('BMGR3616__er', (0, 0.7411134633688338)),
             ('PKS_C204__cytop', (0, 0.706155916711654)),
             ('EX_C00188__dra', (0, 1)),
             ('TO0000017tm__mitmem', (0, 0.6340927836623996)),
             ('EX_C05839__dra', (0, 1)),
             ('R09503_hn438__lum', (0, 0.7060554238610321)),
             ('BMGR0422__chlo', (0, 0.6372038720346583)),
             ('TO0000309t__plas', (0, 0.5288593720600674)),
             ('R04537__cytop', (0, 0.8418549010860761)),

In [62]:
c3.fluxes['R00200__mito']

0.0

In [63]:
p3.fluxes['R00200__mito']

0.0

In [54]:
threshold = 0.1
flux_change = {}
for key, value in c3.fluxes.items():
    if abs(value + p3.fluxes[key]) != 0:
        flux_change[key] = (value - p3.fluxes[key]) / abs(value + p3.fluxes[key])
fx = pd.DataFrame.from_dict(data=flux_change, orient='index', columns=['Flux_change'])
as_dict = fx.loc[(fx.Flux_change >= threshold) | (fx.Flux_change <= -threshold)].to_dict(orient='index')
for key, value in as_dict.items():
    as_dict[key] = round(value['Flux_change'], 3)
as_dict

{}

In [69]:
def flux_capacity(constraints):
    res = {}
    for key, value in constraints.items():
        res[key] = max(abs(value[0]), abs(value[1]))
    return res

In [72]:
fc_c3 = flux_capacity(c3.get_constraints())

In [84]:
fc_n5 = flux_capacity(n5.get_constraints())

In [86]:
for key, value in fc_c3.items():
    if fc_n5[key] + value != 0:
        fc = (value - fc_n5[key]) / abs(fc_n5[key] + value)
        if abs(fc) > 0.3:
            print(key, fc)

TZ2900003t__plas -1.0


In [89]:
fc_c3['R00200__cytop']

0.7467616033720962